**In this Notebook we will design the full graph using 3 tools: search_tool, RAG tool, and SQL-agent for travel database**

In [1]:
import os
from dotenv import load_dotenv
from pyprojroot import here
load_dotenv()

True

Set the environment variables

In [2]:
os.environ['TAVILY_API_KEY'] = os.getenv("TAVILY_API_KEY")

### **1. initialize the Tools**

**1.1 RAG tool design**

In [3]:
import chromadb
import os
import spacy
from langchain_core.tools import tool


EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL_NAME")
VECTORDB_DIR = "data/airline_policy_vectordb"
K = 2

@tool
def lookup_policy(query: str) -> str:
    """Consult the company policies to check whether certain options are permitted."""
    
    client = chromadb.PersistentClient(path=str(here(VECTORDB_DIR)))
    collection = client.get_or_create_collection(name="rag-chroma")

    nlp = spacy.load(EMBEDDING_MODEL)
    query_embedding = nlp(query).vector

    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=K,
        include=["documents"]
    )
    return "\n\n".join(results["documents"][0])

print(lookup_policy)

name='lookup_policy' description='Consult the company policies to check whether certain options are permitted.' args_schema=<class 'langchain_core.utils.pydantic.lookup_policy'> func=<function lookup_policy at 0x78236d69d6c0>


Test the RAG tool

In [4]:
from pprint import pprint
pprint(lookup_policy.invoke("can I cancel my ticket?"))

('hour cancellation policy that allows passengers to cancel their flights '
 'within 24 hours of booking at +1-877-507-7341\n'
 'without penalty. This policy applies to all fare types, including '
 'non-refundable tickets. If you cancel your Swiss Airlines\n'
 "flight within 24 hours of booking, you'll receive a full refund of your "
 'ticket price.\n'
 'How to Cancel Swiss Airlines Flight within 24 Hours? If you need to cancel '
 'your Swiss Airlines flight within 24 hours of\n'
 'booking, you can do so easily online. Here are the steps to follow:\n'
 'Go to Swiss Airlines\' website and click on the "Manage your bookings" tab. '
 'Enter your booking reference number and last\n'
 'name to access your booking. Select the flight you want to cancel and click '
 'on "Cancel flight." Confirm your cancellation\n'
 "and you'll receive a full refund of your ticket price. If you booked your "
 "Swiss Airlines flight through a travel agent, you'll\n"
 'need to contact them directly to cancel you

**1.2 Search tool design**

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_search = TavilySearchResults(max_results=2)

@tool
def search_tool(query: str) -> str:
    """Search the web using Tavily and return the results as a string."""
    results = tavily_search.invoke(query)
    # Optionally, format the results as a string for LLM consumption
    if isinstance(results, list):
        return "\n\n".join(
            f"{item.get('title', '')}\n{item.get('url', '')}\n{item.get('content', '')}"
            for item in results
        )
    return str(results)

/tmp/ipykernel_62715/2142490942.py:3: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tavily_search = TavilySearchResults(max_results=2)


Test the Search Tool

In [6]:
search_tool.invoke("What's a 'node' in LangGraph?")

'Introduction to LangGraph: A Beginner\'s Guide - Medium\nhttps://medium.com/@cplog/introduction-to-langgraph-a-beginners-guide-14f9be027141\nStateful Graph: LangGraph revolves around the concept of a stateful graph, where each node in the graph represents a step in your computation, and the graph maintains a state that is passed around and updated as the computation progresses.\n   Nodes: Nodes are the building blocks of your LangGraph. Each node represents a function or a computation step. You define nodes to perform specific tasks, such as processing input, making decisions, or interacting with external APIs.\n\nstate graph node - GitHub Pages\nhttps://langchain-ai.github.io/langgraph/concepts/low_level/\nclass State(MessagesState):\n    documents: list[str]\n```\n\nNodes¶\n---------------------------------------------------------------------------------------------\n\nIn LangGraph, nodes are typically python functions (sync or async) where the first positional argument is the state

**1.3 SQL agent tool design**

In [7]:
from langchain_community.utilities import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.prompts import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter

In [8]:
import re

def extract_sql_query(text: str) -> str:
    """
    Extract a clean SQL query from a possibly formatted or prefixed LLM output.
    Handles code blocks, 'SQLQuery:' prefix, and various SQL command types.
    """
    text = text.strip()

    # Step 1: Remove code block markers
    if text.startswith("```"):
        lines = text.splitlines()
        # Remove first/last lines if they're ``` or ```sql
        if lines[0].startswith("```") and lines[-1].startswith("```"):
            text = "\n".join(lines[1:-1]).strip()

    # Step 2: Remove known prefixes like "SQLQuery:"
    text = re.sub(r"^(SQLQuery:|Query:)\s*", "", text, flags=re.IGNORECASE)

    # Step 3: Extract only the actual SQL statement (any type)
    match = re.search(
        r"(SELECT|INSERT|UPDATE|DELETE|PRAGMA|CREATE|DROP|ALTER|DESCRIBE|SHOW)\s.+",
        text,
        flags=re.IGNORECASE | re.DOTALL,
    )
    if match:
        query = match.group(0).strip()
        # Remove trailing backticks or semicolons
        return query.rstrip(";`").strip()

    # Step 4: If nothing matched, return original (but cleaned)
    return text


**SQL agent chain**

In [9]:
sqldb_directory = here("data/travel.sqlite")

sql_llm = ChatGoogleGenerativeAI(
    model=os.getenv("MODEL_NAME"),
    temperature=os.getenv("TEMPERATURE"),
    google_api_key=os.getenv("GEMINI_API_KEY")
)

system_role = """Given the following user question, corresponding SQL query, and SQL result, answer the user question.\n
    Question: {question}\n
    SQL Query: {query}\n
    SQL Result: {result}\n
    Answer:
    """
db = SQLDatabase.from_uri(
    f"sqlite:///{sqldb_directory}")

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(
    sql_llm, db)
clean_query_output = RunnableLambda(extract_sql_query)
answer_prompt = PromptTemplate.from_template(
    system_role)


answer = answer_prompt | sql_llm | StrOutputParser()
chain = (
    RunnablePassthrough.assign(raw_query=write_query).assign(
        query=itemgetter("raw_query") | clean_query_output
    ).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

@tool
def query_sqldb(query: str) -> str:
    """Query the Swiss Airline SQL Database and access all the company's information. Input should be a search query."""
    response = chain.invoke({"question": query})
    return response

/tmp/ipykernel_62715/3863574311.py:18: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


In [10]:
# Test the chain
message = "How many tables do I have in the database? and what are their names?"
response = query_sqldb.invoke(message)
print(response)

You have 11 tables in the database. Their names are: aircrafts_data, airports_data, boarding_passes, bookings, flights, seats, ticket_flights, tickets, car_rentals, hotels, and trip_recommendations.


**Define Your Tools as JSON Schemas**

In [11]:
from pydantic import create_model
import inspect

def jsonschema(f):
    kw = {n: (o.annotation, ... if o.default == inspect.Parameter.empty else o.default)
          for n, o in inspect.signature(f).parameters.items()}
    s = create_model(f'Input for `{f.__name__}`', **kw).model_json_schema()  # Use model_json_schema for Pydantic v2+
    return dict(name=f.__name__, description=f.__doc__, parameters=s)

**Wrap up the tools into a list**

In [12]:
# For each tool, generate its schema
lookup_policy_json = jsonschema(lookup_policy.func)
query_sqldb_json = jsonschema(query_sqldb.func)
search_tool_json = jsonschema(search_tool.func)
tools = [lookup_policy_json, query_sqldb_json, search_tool_json]

### **2. Initialize the Graph State**

Define our StateGraph's state as a typed dictionary containing an append-only list of messages. These messages form the chat history, which is all the state our chatbot needs.

In [13]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages


class State(TypedDict):
    messages: Annotated[list, add_messages]


graph_builder = StateGraph(State)

### **3. Define the Graph Nodes**

**3.1 First node: chatbot**

In [14]:
from fireworks.client import ChatCompletion
from langchain_core.messages import AIMessage

model_name = os.getenv("FIREWORKS_MODEL_NAME")

def chatbot(state):
    # Convert state["messages"] to OpenAI-style messages
    messages = []
    for msg in state["messages"]:
        if isinstance(msg, dict):
            # Already in OpenAI format
            messages.append(msg)
        else:
            # Convert from LangChain message to dict
            if hasattr(msg, 'model_dump'):
                msg_dict = msg.model_dump()
            else:
                msg_dict = msg.__dict__
            
            # Filter to only include required fields
            if msg_dict.get('type') == 'ai' or msg_dict.get('role') == 'assistant':
                # Handle assistant messages with tool calls
                converted_msg = {
                    'role': 'assistant',
                    'content': msg_dict.get('content', '')
                }
                
                # Handle tool calls if present
                if 'tool_calls' in msg_dict and msg_dict['tool_calls']:
                    converted_tool_calls = []
                    for tool_call in msg_dict['tool_calls']:
                        if isinstance(tool_call, dict):
                            # Convert from LangChain format to Fireworks format
                            if 'name' in tool_call and 'args' in tool_call:
                                converted_tool_calls.append({
                                    'function': {
                                        'name': tool_call['name'],
                                        'arguments': json.dumps(tool_call['args'])
                                    },
                                    'id': tool_call.get('id', '')
                                })
                            elif 'function' in tool_call:
                                # Already in correct format
                                converted_tool_calls.append(tool_call)
                    
                    if converted_tool_calls:
                        converted_msg['tool_calls'] = converted_tool_calls
                
                messages.append(converted_msg)
                
            elif msg_dict.get('type') == 'human' or msg_dict.get('role') == 'user':
                messages.append({
                    'role': 'user',
                    'content': msg_dict.get('content', '')
                })
            elif msg_dict.get('type') == 'tool' or msg_dict.get('role') == 'tool':
                messages.append({
                    'role': 'tool',
                    'content': msg_dict.get('content', ''),
                    'tool_call_id': msg_dict.get('tool_call_id', '')
                })
    
    response = ChatCompletion.create(
        model=model_name,
        messages=messages,
        functions=tools,
        function_call="auto",
        temperature=0
    )
    
    # Convert the response to LangChain format
    response_msg = response.choices[0].message
    
    
    
    # Convert to LangChain AIMessage
    # Handle tool calls if present
    if hasattr(response_msg, 'tool_calls') and response_msg.tool_calls:
        tool_calls = []
        for tool_call in response_msg.tool_calls:
            
            # Convert Fireworks tool call to LangChain format
            if hasattr(tool_call, 'function'):
                converted_tool_call = {
                    'name': tool_call.function.name,
                    'args': json.loads(tool_call.function.arguments),
                    'id': tool_call.id,
                    'type': 'tool_call'
                }
            else:
                # Already in dict format
                converted_tool_call = tool_call
            
            tool_calls.append(converted_tool_call)
        
        ai_message = AIMessage(
            content=response_msg.content,
            tool_calls=tool_calls
        )
    else:
        ai_message = AIMessage(content=response_msg.content)
    
    return {"messages": [ai_message]}
        
graph_builder.add_node("chatbot", chatbot)

Next, we need to create a function that will run the tools when they are needed. To do this, we'll add the tools to a new node.

In the example below, we'll build a BasicToolNode. This node will check the latest message and, if it contains a request to use a tool, it will run the appropriate tool. This works because many language models (like Anthropic, OpenAI, and Google Gemini) support tool usage.

**3.2 Second node: BasicToolNode that runs the appropriate tool based on the primary agent's output**

In [15]:
import json

class BasicToolNode:
    """A node that runs the tools requested in the last AIMessage."""

    def __init__(self, tools: list) -> None:
        # For StructuredTool objects, use .name attribute
        self.tools_by_name = {tool.name: tool for tool in tools}

    def __call__(self, inputs: dict):
        messages = inputs.get("messages", [])
        message = messages[-1]
        outputs = []
        
        for tool_call in message.tool_calls:

            # Handle both dictionary and object formats
            if isinstance(tool_call, dict):
 
                # Check if it's the new format (name/args directly) or old format (function nested)
                if "function" in tool_call:
                    # Old format: {'function': {'name': '...', 'arguments': '...'}, 'id': '...'}
                    tool_name = tool_call["function"]["name"]
                    tool_args = json.loads(tool_call["function"]["arguments"])
                    tool_call_id = tool_call["id"]
                else:
                    # New format: {'name': '...', 'args': {...}, 'id': '...'}
                    tool_name = tool_call["name"]
                    tool_args = tool_call["args"]  # Already a dict, no need to json.loads
                    tool_call_id = tool_call["id"]
            else:
                # Object format (original Fireworks object)
                tool_name = tool_call.function.name
                tool_args = json.loads(tool_call.function.arguments)
                tool_call_id = tool_call.id
            
            # Get the tool and invoke it
            if tool_name in self.tools_by_name:
                tool = self.tools_by_name[tool_name]
                result = tool.invoke(tool_args)
            else:
                result = f"Unknown tool: {tool_name}"
            
            outputs.append({
                "role": "tool",
                "content": json.dumps(result),
                "name": tool_name,
                "tool_call_id": tool_call_id,
            })
        return {"messages": outputs}
                
# Create the tool node with your StructuredTool objects
tool_node = BasicToolNode(
    tools=[
        search_tool, lookup_policy, query_sqldb
    ])
graph_builder.add_node("tools", tool_node)

### **4. Define the entry point and graph edges**

**Aproach 1**

In [16]:
from typing import Literal


def route_tools(
    state: State,
) -> Literal["tools", "__end__"]:
    """
    Use in the conditional_edge to route to the ToolNode if the last message
    has tool calls. Otherwise, route to the end.
    """
    if isinstance(state, list):
        ai_message = state[-1]
    elif messages := state.get("messages", []):
        ai_message = messages[-1]
    else:
        raise ValueError(f"No messages found in input state to tool_edge: {state}")
    if hasattr(ai_message, "tool_calls") and len(ai_message.tool_calls) > 0:
        return "tools"
    return "__end__"


# The `tools_condition` function returns "tools" if the chatbot asks to use a tool, and "__end__" if
# it is fine directly responding. This conditional routing defines the main agent loop.
graph_builder.add_conditional_edges(
    "chatbot",
    route_tools,
    # The following dictionary lets you tell the graph to interpret the condition's outputs as a specific node
    # It defaults to the identity function, but if you
    # want to use a node named something else apart from "tools",
    # You can update the value of the dictionary to something else
    # e.g., "tools": "my_tools"
    {"tools": "tools", "__end__": "__end__"},
)
# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

**Approach 2**

In [17]:
# from langgraph.graph import END, MessagesState
# from typing import Literal

# # Define the function that determines whether to continue or not
# def should_continue(state: MessagesState) -> Literal["tools", END]:
#     messages = state['messages']
#     last_message = messages[-1]
#     # If the LLM makes a tool call, then we route to the "tools" node
#     if last_message.tool_calls:
#         return "tools"
#     # Otherwise, we stop (reply to the user)
#     return END

# graph_builder.add_conditional_edges(
#     "chatbot",
#     should_continue,
#     ["tools", END],
# )
# # Any time a tool is called, we return to the chatbot to decide the next step
# graph_builder.add_edge("tools", "chatbot")
# graph_builder.add_edge(START, "chatbot")

### **5. Compile the graph**

- In this step, we can add a memory to our graph as well.

In [18]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)

**5.1 Plot the compiled graph**

In [19]:
# from IPython.display import Image, display

# try:
#     display(Image(graph.get_graph().draw_mermaid_png()))
# except Exception:
#     # This requires some extra dependencies and is optional
#     print("didn't generate the image")

# Print the graph structure as text
# print("Graph Structure:")
# print(graph.get_graph())

**Save the graph image**

In [20]:
# save = False

# if save:
#     from PIL import Image as PILImage
#     import io
#     # Assuming graph.get_graph().draw_mermaid_png() returns PNG binary data
#     try:
#         # Generate the PNG image from the graph
#         png_data = graph.get_graph().draw_mermaid_png()
        
#         # Convert the binary data into an image
#         img = PILImage.open(io.BytesIO(png_data))
        
#         # Save the image locally with 300 DPI
#         img.save('output_image1.png', 'PNG', dpi=(300, 300))
        
#         print("Image saved successfully with 300 DPI.")
#     except Exception as e:
#         print(f"Error: {e}")

### **6. Execute the graph**

In [21]:
config = {"configurable": {"thread_id": "1"}}

##### **First query**

Approach 1: Print all the steps the the system goes through ti get the final output

In [22]:
user_input = "Hi there! My name is Zubair."

# The config is the **second positional argument** to stream() or invoke()!

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi there! My name is Zubair.


ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x782349345cd0>


================================== Ai Message ==================================

<think>
Okay, the user said, "Hi there! My name is Zubair." Let me think about how to respond.

First, I should acknowledge their greeting. A simple "Hello" or "Hi" would work. Then, maybe ask how I can assist them. Since they mentioned their name, maybe include that in the response to be friendly. 

Wait, but do I need to use any tools here? The functions provided are for looking up policies, querying a database, or searching the web. The user hasn't asked a question that requires any of those tools yet. They're just introducing themselves. 

So, no function calls are needed at this point. Just a polite response. I'll go with something like, "Hello Zubair! How can I assist you today?" That's friendly and opens the door for them to ask questions. 

I should make sure not to use any markdown and keep it natural. Alright, that should do it.
</think>

Hello Zubair! How can I assist you today?


Approach 2: Just print the final output

In [23]:
from langchain_core.messages import HumanMessage
user_input = "Hi there! My name is Zubair."

# Use the Runnable
final_state = graph.invoke(
    {"messages": [HumanMessage(content=user_input)]},
    config=config
)
final_state["messages"][-1].content

ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x782349345cd0>


'<think>\nOkay, the user said "Hi there! My name is Zubair." again. They might just be testing or trying to get a response. I should acknowledge their name again and ask how I can help. Let me make sure to keep it friendly and open-ended. Maybe they have a specific request but aren\'t sure how to phrase it. I\'ll stay welcoming and ready to assist.\n</think>\n\nHello Zubair! How can I assist you today? 😊'

#### **Second query**

In [24]:
user_input = "Can I cancel my ticket 10 hours before the flight?"

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Can I cancel my ticket 10 hours before the flight?


ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7823411aeab0>


================================== Ai Message ==================================

<think>
Okay, the user is asking if they can cancel their ticket 10 hours before the flight. I need to check the company's policies regarding ticket cancellations. The available tools include lookup_policy, which is designed to consult company policies. So, I should call the lookup_policy function with the query "Can I cancel my ticket 10 hours before the flight?" That should retrieve the relevant policy information. I don't think I need to use the other tools like query_sqldb or search_tool here because the question is about policy, not database data or web search. Let me make sure the parameters are correct. The function requires a "query" parameter, which I have. Alright, time to make the tool call.
</think>
Tool Calls:
  lookup_policy (call_SbhixNVhChOjNKcV2jb3owNf)
 Call ID: call_SbhixNVhChOjNKcV2jb3owNf
  Args:
    query: Can I cancel my ticket 10 hours before the flight?
===========================

ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x7823205781d0>


================================== Ai Message ==================================

<think>
Okay, let me process the information from the tool response. The user asked if they can cancel their ticket 10 hours before the flight. The tool response mentions a 24-hour cancellation policy without penalty, but the user is asking about 10 hours. The key point here is that the policy allows cancellation within 24 hours of booking, not necessarily 10 hours before the flight. Wait, the response says "within 24 hours of booking," which might be different from 10 hours before departure. The user might be confused between the time since booking versus time before the flight. I need to clarify that the 24-hour window is from when the ticket was purchased, not the departure time. So if they booked the ticket 10 hours before the flight, they might still be within the 24-hour window. But if the 10 hours is before the flight, but the booking was done more than 24 hours ago, then it might not apply. The an

#### **Third query**

In [25]:
user_input = "Right now Harris vs. Trump Presidential Debate is being boradcasted. I want the youtube link to this debate"

events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Right now Harris vs. Trump Presidential Debate is being boradcasted. I want the youtube link to this debate


ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x78232057ab40>


================================== Ai Message ==================================

<think>
Okay, the user is asking for a YouTube link to the current Harris vs. Trump presidential debate. Let me think about how to handle this.

First, I need to check if I have a function that can search for YouTube videos. The available tools are lookup_policy, query_sqldb, and search_tool. The search_tool uses Tavily to search the web. So, I can use search_tool to find the YouTube link.

But wait, the user might be expecting a direct link. However, since I can't browse the internet in real-time, I need to use the search_tool to get the latest information. Let me structure the query properly. The user is asking for the YouTube link, so the search query should be something like "Harris vs. Trump debate YouTube live stream" or similar.

I should call the search_tool with that query. Once I get the results, I can extract the link from the search results. But I need to make sure the link is accurate and fro

ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x78232057a4b0>


================================== Ai Message ==================================

<think>
Okay, the user asked for the YouTube link to the Harris vs. Trump debate. I used the search_tool to find it. The first result has a link: https://www.youtube.com/watch?v=lAH6KUOiSB8. But I should check if that's the correct one. The title mentions a recap of the debate from September 10, which might be the latest one. The second result is from C-SPAN, but it's a simulcast link. However, the user specifically asked for a YouTube link, so the first result is more relevant. I should present that link and mention that it's a recap. Also, note that the debate might be available on other platforms like C-SPAN. Make sure to inform the user about the link and possible alternatives.
</think>

Here's the YouTube link to the **Harris vs. Trump presidential debate recap** (as of the latest available search results):

🔗 [Watch the Debate Recap on YouTube](https://www.youtube.com/watch?v=lAH6KUOiSB8)

### Notes

#### **Fourth query**

In [26]:
user_input = "How many tables do I have in the database? and what are their names?"
events = graph.stream(
    {"messages": [{"role": "user", "content": user_input}]}, config, stream_mode="values"
)
for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

How many tables do I have in the database? and what are their names?


ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x78232057ae40>


================================== Ai Message ==================================

<think>
Okay, the user is asking how many tables are in the database and what their names are. Let me check the tools available.

Looking at the tools provided, there's the `query_sqldb` function which allows querying the Swiss Airline SQL Database. The description says it can access all the company's information. So, to get the number of tables and their names, I should use this function.

But wait, the user didn't specify any particular database schema or table structure. The tools don't include a function to list all tables directly. However, the `query_sqldb` might be able to run a query that retrieves table information. For example, in SQL, you can query the information_schema.tables to get table names and count them.

So, I need to construct a query that selects the count of tables and their names. The exact SQL syntax might depend on the database system, but since it's a generic SQL database, using

ERROR:asyncio:Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x78232061fbc0>


================================== Ai Message ==================================

<think>
Okay, the user asked how many tables are in the database and their names. I used the `query_sqldb` tool with a SQL query to fetch the table information. The response from the tool listed 11 tables with their names.

Now, I need to present this information clearly. Let me check if the response includes all the tables correctly. The list is: aircrafts_data, airports_data, boarding_passes, bookings, flights, seats, ticket_flights, tickets, car_rentals, hotels, and trip_recommendations. That's 11 tables. 

I should confirm the count and list them out in a user-friendly way. Also, maybe offer further assistance in case they need more details about specific tables. Keep the answer concise and helpful.
</think>

The database contains **11 tables** with the following names:  
1. `aircrafts_data`  
2. `airports_data`  
3. `boarding_passes`  
4. `bookings`  
5. `flights`  
6. `seats`  
7. `ticket_flights`  

#### **History**

In [27]:
event

{'messages': [HumanMessage(content='Hi there! My name is Zubair.', additional_kwargs={}, response_metadata={}, id='01c7ecc5-4af9-4651-a035-e6d6fb72e72c'),
  AIMessage(content='<think>\nOkay, the user said, "Hi there! My name is Zubair." Let me think about how to respond.\n\nFirst, I should acknowledge their greeting. A simple "Hello" or "Hi" would work. Then, maybe ask how I can assist them. Since they mentioned their name, maybe include that in the response to be friendly. \n\nWait, but do I need to use any tools here? The functions provided are for looking up policies, querying a database, or searching the web. The user hasn\'t asked a question that requires any of those tools yet. They\'re just introducing themselves. \n\nSo, no function calls are needed at this point. Just a polite response. I\'ll go with something like, "Hello Zubair! How can I assist you today?" That\'s friendly and opens the door for them to ask questions. \n\nI should make sure not to use any markdown and keep 